In [5]:
import numpy as np
import numpy.linalg as la
import importlib
from tqdm import tqdm
import bandit as bd
importlib.reload(bd)
import h5py
from matplotlib import pyplot as plt
from matplotlib import figure

In [6]:
data=np.load('data/data_exp_N20_T10000_beta1.npy')
T=1000
N=20
k=7
beta=1
dtype=f'exp_beta_{beta}'
# r_hedge=bd.hedge(data,T,N,k)
r_sp=bd.bandit_sparse_estimator(data,T,N,k)
r_lstsq=bd.bandit_least_sq_estimator(data,T,N,k)
r_ips=bd.bandit_ips_estimator(data,T,N,k)

  0%|          | 0/10 [00:00<?, ?it/s]

IndexError: index 16 is out of bounds for axis 0 with size 16

In [43]:
store=h5py.File(f'output/output_{dtype}_files_{N}_cache_{k}_time_{T}.h5','w')
store.create_dataset('data',data=data)
store['data'].attrs['num_files']=N
store['data'].attrs['cache_size']=k
store['data'].attrs['time']=T
store['regret/sparse_estimator']=r_sp
store['regret/hedge']=r_hedge
store['regret/least_sq_estimator']=r_lstsq
store['regret/ips_estimator']=r_ips
store.close()

In [ ]:
with h5py.File('output/output_exp_beta_0.5_files_20_cache_7_time_50000.h5','r') as store:
    plt.figure(dpi=200)
    plt.plot(store['regret/hedge'], label='hedge')
    plt.plot(store['regret/sparse_estimator'], label='sparse_estimator')
    plt.plot(store['regret/least_sq_estimator'], label='least_sq_estimator')
    plt.plot(store['regret/ips_estimator'], label='ips_estimator')
    plt.legend()

In [122]:
5//2

2